In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)


<IPython.core.display.Javascript object>

In [48]:
import pandas as pd
import numpy as np
import os
import sys

# input_data_folder = "./data/trafficFines/"
# output_data_folder = "./data/trafficFines/labeled_logs_csv_processed"
# filenames = ["Road_Traffic_Fine_Management_Process.csv"]

case_id_col = "Case ID"
activity_col = "Activity"
resource_col = "Resource"
timestamp_col = "Complete Timestamp"
label_col = "label"
pos_label = "deviant"
neg_label = "regular"

freq_threshold = 10

# features for classifier
dynamic_cat_cols = ["Activity", "time_to_event_m"]
static_cat_cols = []
dynamic_num_cols = [
    "timesincelastevent",
    "timesincecasestart",
    "timesincemidnight",
    "event_nr",
    "month",
    "weekday",
    "hour",
    "open_cases",
]
static_num_cols = []

static_cols = static_cat_cols + static_num_cols + [case_id_col]
dynamic_cols = dynamic_cat_cols + dynamic_num_cols + [timestamp_col]
cat_cols = dynamic_cat_cols + static_cat_cols

<IPython.core.display.Javascript object>

In [47]:
data_all.columns

Index(['Activity', 'lifecycle:transition', 'Complete Timestamp', 'Case ID',
       'timesincemidnight', 'month', 'weekday', 'hour', 'timesincelastevent',
       'timesincecasestart', 'event_nr', 'time_to_event_m', 'case_length',
       'open_cases', 'treatment'],
      dtype='object')

<IPython.core.display.Javascript object>

In [3]:
import pm4py

log = pm4py.read_xes("./data/trafficFines/Road_Traffic_Fine_Management_Process.xes.gz")


/home/mshoush/anaconda3/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:76: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [4]:
data = pm4py.convert_to_dataframe(log)


<IPython.core.display.Javascript object>

In [5]:
data.columns

Index(['amount', 'org:resource', 'dismissal', 'concept:name', 'vehicleClass',
       'totalPaymentAmount', 'lifecycle:transition', 'time:timestamp',
       'article', 'points', 'case:concept:name', 'expense', 'notificationType',
       'lastSent', 'paymentAmount', 'matricola'],
      dtype='object')

<IPython.core.display.Javascript object>

In [6]:
data = data.rename(
    columns={
        "case:concept:name": "Case ID",
        "concept:name": "Activity",
        "org:resource": "Resource",
        "time:timestamp": "Complete Timestamp",
    }
)
data

,amount,Resource,dismissal,Activity,vehicleClass,totalPaymentAmount,lifecycle:transition,Complete Timestamp,article,points,Case ID,expense,notificationType,lastSent,paymentAmount,matricola
0,35.0,561,NIL,Create Fine,A,0.0,complete,2006-07-23 22:00:00+00:00,157.0,0.0,A1,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-04 23:00:00+00:00,NaN,NaN,A1,11.00,NaN,NaN,NaN,NaN
2,35.0,561,NIL,Create Fine,A,0.0,complete,2006-08-01 22:00:00+00:00,157.0,0.0,A100,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-11 23:00:00+00:00,NaN,NaN,A100,11.00,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2007-01-14 23:00:00+00:00,NaN,NaN,A100,NaN,P,P,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561465,131.0,25,NIL,Create Fine,A,0.0,complete,2002-09-06 22:00:00+00:00,142.0,0.0,V9999,NaN,NaN,NaN,NaN,NaN
561466,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2002-10-24 22:00:00+00:00,NaN,NaN,V9999,15.16,NaN,NaN,NaN,NaN
561467,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2002-11-03 23:00:00+00:00,NaN,NaN,V9999,NaN,P,N,NaN,NaN
561468,262.0,NaN,NaN,Add penalty,NaN,NaN,complete,2003-01-02 23:00:00+00:00,NaN,NaN,V9999,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [7]:
data["Complete Timestamp"] = pd.to_datetime(
    data["Complete Timestamp"], infer_datetime_format=True,
)

data["Complete Timestamp"] = data["Complete Timestamp"].dt.tz_convert(None)

<IPython.core.display.Javascript object>

In [8]:
data = data.dropna(axis="columns")
data

,Activity,lifecycle:transition,Complete Timestamp,Case ID
0,Create Fine,complete,2006-07-23 22:00:00,A1
1,Send Fine,complete,2006-12-04 23:00:00,A1
2,Create Fine,complete,2006-08-01 22:00:00,A100
3,Send Fine,complete,2006-12-11 23:00:00,A100
4,Insert Fine Notification,complete,2007-01-14 23:00:00,A100
...,...,...,...,...
561465,Create Fine,complete,2002-09-06 22:00:00,V9999
561466,Send Fine,complete,2002-10-24 22:00:00,V9999
561467,Insert Fine Notification,complete,2002-11-03 23:00:00,V9999
561468,Add penalty,complete,2003-01-02 23:00:00,V9999


<IPython.core.display.Javascript object>

In [9]:
# # impute missing values
# grouped = data.sort_values(timestamp_col, ascending=True, kind="mergesort").groupby(
#     case_id_col
# )
# for col in static_cols + dynamic_cols:
#     data[col] = grouped[col].transform(lambda grp: grp.fillna(method="ffill"))

# data[cat_cols] = data[cat_cols].fillna("missing")
# data = data.fillna(0)
# data

<IPython.core.display.Javascript object>

In [10]:
# discard incomplete cases
last_events = (
    data.sort_values([timestamp_col], ascending=True, kind="mergesort")
    .groupby(case_id_col)
    .last()[activity_col]
)
incomplete_cases = last_events.index[last_events == "Send Fine"]
data = data[~data[case_id_col].isin(incomplete_cases)]

# discard incomplete cases
last_events_time_to_event = (
    data.sort_values([timestamp_col], ascending=True, kind="mergesort")
    .groupby(case_id_col)
    .last()[activity_col]
)
list(set(last_events_time_to_event))

['Send Appeal to Prefecture',
 'Receive Result Appeal from Prefecture',
 'Send for Credit Collection',
 'Notify Result Appeal to Offender',
 'Appeal to Judge',
 'Payment']

<IPython.core.display.Javascript object>

In [11]:
data

,Activity,lifecycle:transition,Complete Timestamp,Case ID
2,Create Fine,complete,2006-08-01 22:00:00,A100
3,Send Fine,complete,2006-12-11 23:00:00,A100
4,Insert Fine Notification,complete,2007-01-14 23:00:00,A100
5,Add penalty,complete,2007-03-15 23:00:00,A100
6,Send for Credit Collection,complete,2009-03-29 22:00:00,A100
...,...,...,...,...
561465,Create Fine,complete,2002-09-06 22:00:00,V9999
561466,Send Fine,complete,2002-10-24 22:00:00,V9999
561467,Insert Fine Notification,complete,2002-11-03 23:00:00,V9999
561468,Add penalty,complete,2003-01-02 23:00:00,V9999


<IPython.core.display.Javascript object>

In [12]:
data[data["Case ID"] == "A100"]

,Activity,lifecycle:transition,Complete Timestamp,Case ID
2,Create Fine,complete,2006-08-01 22:00:00,A100
3,Send Fine,complete,2006-12-11 23:00:00,A100
4,Insert Fine Notification,complete,2007-01-14 23:00:00,A100
5,Add penalty,complete,2007-03-15 23:00:00,A100
6,Send for Credit Collection,complete,2009-03-29 22:00:00,A100


<IPython.core.display.Javascript object>

In [15]:
def extract_timestamp_features(group):

    group = group.sort_values(timestamp_col, ascending=True, kind="mergesort")

    tmp = group[timestamp_col] - group[timestamp_col].shift(-1)
    tmp = tmp.fillna(pd.Timedelta(seconds=0))
    group["timesincelastevent"] = tmp.apply(
        lambda x: float(x / np.timedelta64(1, "m"))
    )  # m is for minutes

    tmp = group[timestamp_col] - group[timestamp_col].iloc[-1]
    tmp = tmp.fillna(pd.Timedelta(seconds=0))
    group["timesincecasestart"] = tmp.apply(
        lambda x: float(x / np.timedelta64(1, "m"))
    )  # m is for minutes

    group["event_nr"] = range(1, len(group) + 1)

    try:
        # print("1")
        last_e = pd.to_datetime(
            group[group[activity_col].isin(list(set(last_events_time_to_event)))][
                timestamp_col
            ].values[0]
        )
        # print("2", last_e)
        group["time_to_event_m"] = (
            last_e - group[timestamp_col]
        ).dt.total_seconds() / 60.0
        # print("3", group["time_to_event_m"])
        # break
        group["time_to_event_m"] = group["time_to_event_m"].fillna(0)
        #     group["time_to_event_days"] = group["time_to_event_m"] / 1440
        #     group["time_to_event_wks"] = group["time_to_event_days"] / 7
        # print("4")
        group["case_length"] = group["event_nr"].max()

        return group
    except:
        print(f"False, continue, {set(group['Case ID'])}")

    return group


def get_open_cases(date):
    return sum(
        (dt_first_last_timestamps["start_time"] <= date)
        & (dt_first_last_timestamps["end_time"] > date)
    )


def check_if_activity_exists(group, activity, cut_from_idx=False):
    relevant_activity_idxs = np.where(group[activity_col] == activity)[0]
    if len(relevant_activity_idxs) > 0:
        idx = relevant_activity_idxs[0]
        group[label_col] = pos_label
        if cut_from_idx:
            return group[:idx]
        else:
            return group
    else:
        group[label_col] = neg_label
        return group


# for filename in filenames:
print("Starting...")
sys.stdout.flush()
data = data  # pd.read_csv(os.path.join(input_data_folder,filename), sep=";")

data.rename(columns=lambda x: x.replace("(case) ", ""), inplace=True)

# discard incomplete cases
# last_events = (
#     data.sort_values([timestamp_col], ascending=True, kind="mergesort")
#     .groupby(case_id_col)
#     .last()["Activity"]
# )
# incomplete_cases = last_events.index[last_events == "Send Fine"]
# data = data[~data[case_id_col].isin(incomplete_cases)]

# add event duration
data[timestamp_col] = pd.to_datetime(data[timestamp_col])
data["timesincemidnight"] = (
    data[timestamp_col].dt.hour * 60 + data[timestamp_col].dt.minute
)
data["month"] = data[timestamp_col].dt.month
data["weekday"] = data[timestamp_col].dt.weekday
data["hour"] = data[timestamp_col].dt.hour

# add features extracted from timestamp
print("Extracting timestamp features...")
sys.stdout.flush()
data = data.groupby(case_id_col).apply(extract_timestamp_features)

# add inter-case features
print("Extracting open cases...")
sys.stdout.flush()
data = data.sort_values([timestamp_col], ascending=True, kind="mergesort")
dt_first_last_timestamps = data.groupby(case_id_col)[timestamp_col].agg([min, max])
dt_first_last_timestamps.columns = ["start_time", "end_time"]
data["open_cases"] = data[timestamp_col].apply(get_open_cases)

# # impute missing values
# grouped = data.sort_values(timestamp_col, ascending=True, kind="mergesort").groupby(
#     case_id_col
# )
# for col in static_cols + dynamic_cols:
#     data[col] = grouped[col].transform(lambda grp: grp.fillna(method="ffill"))

# data[cat_cols] = data[cat_cols].fillna("missing")
# data = data.fillna(0)

# # set infrequent factor levels to "other"
# for col in cat_cols:
#     if col != activity_col:
#         counts = data[col].value_counts()
#         mask = data[col].isin(counts[counts >= freq_threshold].index)
#         data.loc[~mask, col] = "other"

# data = data[static_cols + dynamic_cols]

# assign class labels

# dt_labeled.to_csv(
#     os.path.join(output_data_folder, "traffic_fines_1.csv"), sep=";", index=False
# )

<IPython.core.display.Javascript object>

In [53]:
print("Assigning class labels...")
sys.stdout.flush()
data = data.sort_values([timestamp_col], ascending=True, kind="mergesort")
dt_labeled = data.groupby(case_id_col).apply(
    check_if_activity_exists, activity="Send for Credit Collection", cut_from_idx=False
)

Assigning class labels...


<IPython.core.display.Javascript object>

In [40]:
def check_for_penalty(g):
    if "Add penalty" in set(g["Activity"]):
        # print("True")
        g["treatment"] = "noTreat"
    else:
        # print("FALSE")
        g["treatment"] = "treat"

    return g

<IPython.core.display.Javascript object>

In [58]:
dt_labeled

,Activity,lifecycle:transition,Complete Timestamp,Case ID,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,time_to_event_m,case_length,open_cases,treatment,event,label
0,Create Fine,complete,1999-12-31 23:00:00,S38735,1380,12,4,23,-152580.0,-1195140.0,1,1195140.0,5,1,noTreat,True,deviant
1,Create Fine,complete,2000-01-01 23:00:00,S44306,1380,1,5,23,-63360.0,-668100.0,1,668100.0,5,5,noTreat,True,deviant
2,Create Fine,complete,2000-01-01 23:00:00,S49055,1380,1,5,23,-63360.0,-344100.0,1,149700.0,6,5,noTreat,True,regular
3,Create Fine,complete,2000-01-01 23:00:00,S49056,1380,1,5,23,-63360.0,-1193700.0,1,1193700.0,5,5,noTreat,True,deviant
4,Create Fine,complete,2000-01-01 23:00:00,S49057,1380,1,5,23,-63360.0,-1193700.0,1,1193700.0,5,5,noTreat,True,deviant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519580,Payment,complete,2013-06-17 22:00:00,S194560,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True,regular
519581,Payment,complete,2013-06-17 22:00:00,S194611,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True,regular
519582,Payment,complete,2013-06-17 22:00:00,S194656,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True,regular
519583,Payment,complete,2013-06-17 22:00:00,S194716,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True,regular


<IPython.core.display.Javascript object>

In [41]:
data = dt_labeled.groupby(case_id_col).apply(check_for_penalty)
data = data.reset_index(drop=True)
data

,Activity,lifecycle:transition,Complete Timestamp,Case ID,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,time_to_event_m,case_length,open_cases,treatment
0,Create Fine,complete,1999-12-31 23:00:00,S38735,1380,12,4,23,-152580.0,-1195140.0,1,1195140.0,5,1,noTreat
1,Create Fine,complete,2000-01-01 23:00:00,S44306,1380,1,5,23,-63360.0,-668100.0,1,668100.0,5,5,noTreat
2,Create Fine,complete,2000-01-01 23:00:00,S49055,1380,1,5,23,-63360.0,-344100.0,1,149700.0,6,5,noTreat
3,Create Fine,complete,2000-01-01 23:00:00,S49056,1380,1,5,23,-63360.0,-1193700.0,1,1193700.0,5,5,noTreat
4,Create Fine,complete,2000-01-01 23:00:00,S49057,1380,1,5,23,-63360.0,-1193700.0,1,1193700.0,5,5,noTreat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519580,Payment,complete,2013-06-17 22:00:00,S194560,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat
519581,Payment,complete,2013-06-17 22:00:00,S194611,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat
519582,Payment,complete,2013-06-17 22:00:00,S194656,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat
519583,Payment,complete,2013-06-17 22:00:00,S194716,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat


<IPython.core.display.Javascript object>

In [50]:
# data_all = data
# # impute missing values
# print("impute missing values...")
# grouped = data_all.sort_values(timestamp_col, ascending=True, kind="mergesort").groupby(
#     case_id_col
# )
# for col in static_cols + dynamic_cols:
#     data[col] = grouped[col].transform(lambda grp: grp.fillna(method="ffill"))

# data_all[cat_cols] = data_all[cat_cols].fillna("missing")
# data_all = data_all.fillna(0)

# # set infrequent factor levels to "other"
# for col in cat_cols:
#     counts = data_all[col].value_counts()
#     mask = data_all[col].isin(counts[counts >= freq_threshold].index)
#     data_all.loc[~mask, col] = "other"

<IPython.core.display.Javascript object>

In [51]:
data_all.columns

Index(['Activity', 'lifecycle:transition', 'Complete Timestamp', 'Case ID',
       'timesincemidnight', 'month', 'weekday', 'hour', 'timesincelastevent',
       'timesincecasestart', 'event_nr', 'time_to_event_m', 'case_length',
       'open_cases', 'treatment'],
      dtype='object')

<IPython.core.display.Javascript object>

In [60]:
dt_labeled = dt_labeled.rename(columns={"Complete Timestamp": "time:timestamp"})
dt_labeled

,Activity,lifecycle:transition,time:timestamp,Case ID,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,time_to_event_m,case_length,open_cases,treatment,event,label
0,Create Fine,complete,1999-12-31 23:00:00,S38735,1380,12,4,23,-152580.0,-1195140.0,1,1195140.0,5,1,noTreat,True,deviant
1,Create Fine,complete,2000-01-01 23:00:00,S44306,1380,1,5,23,-63360.0,-668100.0,1,668100.0,5,5,noTreat,True,deviant
2,Create Fine,complete,2000-01-01 23:00:00,S49055,1380,1,5,23,-63360.0,-344100.0,1,149700.0,6,5,noTreat,True,regular
3,Create Fine,complete,2000-01-01 23:00:00,S49056,1380,1,5,23,-63360.0,-1193700.0,1,1193700.0,5,5,noTreat,True,deviant
4,Create Fine,complete,2000-01-01 23:00:00,S49057,1380,1,5,23,-63360.0,-1193700.0,1,1193700.0,5,5,noTreat,True,deviant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519580,Payment,complete,2013-06-17 22:00:00,S194560,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True,regular
519581,Payment,complete,2013-06-17 22:00:00,S194611,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True,regular
519582,Payment,complete,2013-06-17 22:00:00,S194656,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True,regular
519583,Payment,complete,2013-06-17 22:00:00,S194716,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True,regular


<IPython.core.display.Javascript object>

In [61]:
dataset_name = "trafficFines"
try:
    del data_all["end_time"]
except:
    pass
print("Saving csv file...")
results_dir = "./prepared_data/%s/" % dataset_name
import os

if not os.path.exists(os.path.join(results_dir)):
    os.makedirs(os.path.join(results_dir))

# df.to_csv(results_dir + '/prepared_treatment_outcome_bpic2012.csv', index=False, sep=';')
data_all["event"] = 1
data_all["event"] = data_all["event"].astype("bool")

dt_labeled.to_csv(
    os.path.join(
        results_dir, "prepared_treatment_outcome_time_to_event_%s.csv" % dataset_name
    ),
    index=False,
    sep=";",
)

Saving csv file...


<IPython.core.display.Javascript object>

In [57]:
data_all

,Activity,lifecycle:transition,Complete Timestamp,Case ID,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,time_to_event_m,case_length,open_cases,treatment,event
0,Create Fine,complete,1999-12-31 23:00:00,S38735,1380,12,4,23,-152580.0,-1195140.0,1,1195140.0,5,1,noTreat,True
1,Create Fine,complete,2000-01-01 23:00:00,S44306,1380,1,5,23,-63360.0,-668100.0,1,668100.0,5,5,noTreat,True
2,Create Fine,complete,2000-01-01 23:00:00,S49055,1380,1,5,23,-63360.0,-344100.0,1,149700.0,6,5,noTreat,True
3,Create Fine,complete,2000-01-01 23:00:00,S49056,1380,1,5,23,-63360.0,-1193700.0,1,1193700.0,5,5,noTreat,True
4,Create Fine,complete,2000-01-01 23:00:00,S49057,1380,1,5,23,-63360.0,-1193700.0,1,1193700.0,5,5,noTreat,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519580,Payment,complete,2013-06-17 22:00:00,S194560,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True
519581,Payment,complete,2013-06-17 22:00:00,S194611,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True
519582,Payment,complete,2013-06-17 22:00:00,S194656,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True
519583,Payment,complete,2013-06-17 22:00:00,S194716,1320,6,0,22,0.0,0.0,2,0.0,2,0,treat,True


<IPython.core.display.Javascript object>

In [77]:
case_id_col = "case:concept:name"
activity_col = "concept:name"
resource_col = "Resource"
timestamp_col = "time:timestamp"
label_col = "label"
pos_label = "deviant"
neg_label = "regular"

# treatment col
treatment_col = "treatment"
pos_treatment = "treat"  # do treatment
neg_treatment = "noTreat"  # do not treat


# features for classifier
dynamic_cat_cols = ["Activity", "Resource", "lastSent", "notificationType", "dismissal"]
static_cat_cols = ["article", "vehicleClass"]
dynamic_num_cols = [
    "expense",
    "timesincelastevent",
    "timesincecasestart",
    "timesincemidnight",
    "event_nr",
    "month",
    "weekday",
    "hour",
    "open_cases",
]
static_num_cols = ["amount", "points"]


static_cols = static_cat_cols + static_num_cols + [case_id_col, label_col]
dynamic_cols = dynamic_cat_cols + dynamic_num_cols + [timestamp_col]
cat_cols = dynamic_cat_cols + static_cat_cols
freq_threshold = 10

<IPython.core.display.Javascript object>

In [86]:
import pm4py

log = pm4py.read_xes("./data/trafficFines/Road_Traffic_Fine_Management_Process.xes.gz")
log

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
0,35.0,561,NIL,Create Fine,A,0.0,complete,2006-07-23 22:00:00+00:00,157.0,0.0,A1,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-04 23:00:00+00:00,NaN,NaN,A1,11.00,NaN,NaN,NaN,NaN
2,35.0,561,NIL,Create Fine,A,0.0,complete,2006-08-01 22:00:00+00:00,157.0,0.0,A100,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-11 23:00:00+00:00,NaN,NaN,A100,11.00,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2007-01-14 23:00:00+00:00,NaN,NaN,A100,NaN,P,P,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561465,131.0,25,NIL,Create Fine,A,0.0,complete,2002-09-06 22:00:00+00:00,142.0,0.0,V9999,NaN,NaN,NaN,NaN,NaN
561466,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2002-10-24 22:00:00+00:00,NaN,NaN,V9999,15.16,NaN,NaN,NaN,NaN
561467,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2002-11-03 23:00:00+00:00,NaN,NaN,V9999,NaN,P,N,NaN,NaN
561468,262.0,NaN,NaN,Add penalty,NaN,NaN,complete,2003-01-02 23:00:00+00:00,NaN,NaN,V9999,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [87]:
import pm4py

log = pm4py.read_xes("./data/trafficFines/Road_Traffic_Fine_Management_Process.xes.gz")
log
data = pm4py.convert_to_dataframe(log)
data

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
0,35.0,561,NIL,Create Fine,A,0.0,complete,2006-07-23 22:00:00+00:00,157.0,0.0,A1,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-04 23:00:00+00:00,NaN,NaN,A1,11.00,NaN,NaN,NaN,NaN
2,35.0,561,NIL,Create Fine,A,0.0,complete,2006-08-01 22:00:00+00:00,157.0,0.0,A100,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-11 23:00:00+00:00,NaN,NaN,A100,11.00,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2007-01-14 23:00:00+00:00,NaN,NaN,A100,NaN,P,P,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561465,131.0,25,NIL,Create Fine,A,0.0,complete,2002-09-06 22:00:00+00:00,142.0,0.0,V9999,NaN,NaN,NaN,NaN,NaN
561466,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2002-10-24 22:00:00+00:00,NaN,NaN,V9999,15.16,NaN,NaN,NaN,NaN
561467,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2002-11-03 23:00:00+00:00,NaN,NaN,V9999,NaN,P,N,NaN,NaN
561468,262.0,NaN,NaN,Add penalty,NaN,NaN,complete,2003-01-02 23:00:00+00:00,NaN,NaN,V9999,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [80]:
set(data["concept:name"])

{'Add penalty',
 'Appeal to Judge',
 'Create Fine',
 'Insert Date Appeal to Prefecture',
 'Insert Fine Notification',
 'Notify Result Appeal to Offender',
 'Payment',
 'Receive Result Appeal from Prefecture',
 'Send Appeal to Prefecture',
 'Send Fine',
 'Send for Credit Collection'}

<IPython.core.display.Javascript object>

In [81]:
last_events = (
    data.sort_values(["time:timestamp"], ascending=True, kind="mergesort")
    .groupby("case:concept:name")
    .last()["concept:name"]
)
incomplete_cases = last_events.index[last_events == "Send Fine"]
data = data[~data["case:concept:name"].isin(incomplete_cases)]

<IPython.core.display.Javascript object>

In [82]:
last_events = (
    data.sort_values(["time:timestamp"], ascending=True, kind="mergesort")
    .groupby("case:concept:name")
    .last()["concept:name"]
)
set(last_events)

{'Appeal to Judge',
 'Notify Result Appeal to Offender',
 'Payment',
 'Receive Result Appeal from Prefecture',
 'Send Appeal to Prefecture',
 'Send for Credit Collection'}

<IPython.core.display.Javascript object>

In [69]:
list(set(last_events))

['Send Appeal to Prefecture',
 'Payment',
 'Appeal to Judge',
 'Add penalty',
 'Receive Result Appeal from Prefecture',
 'Notify Result Appeal to Offender']

<IPython.core.display.Javascript object>

In [83]:
data.head()

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
2,35.0,561,NIL,Create Fine,A,0.0,complete,2006-08-01 22:00:00+00:00,157.0,0.0,A100,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-11 23:00:00+00:00,NaN,NaN,A100,11.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2007-01-14 23:00:00+00:00,NaN,NaN,A100,NaN,P,P,NaN,NaN
5,71.5,NaN,NaN,Add penalty,NaN,NaN,complete,2007-03-15 23:00:00+00:00,NaN,NaN,A100,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,Send for Credit Collection,NaN,NaN,complete,2009-03-29 22:00:00+00:00,NaN,NaN,A100,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [ ]:
def extract_timestamp_features_2(group):
    group[timestamp_col] = pd.to_datetime(group[timestamp_col])

    group = group.sort_values(timestamp_col, ascending=False, kind='mergesort')
    
    tmp = group[timestamp_col] - group[timestamp_col].shift(-1)
    tmp = tmp.fillna(pd.Timedelta(seconds=0))
    group["timesincelastevent"] = tmp.apply(lambda x: float(x / np.timedelta64(1, 'm'))) # m is for minutes

    tmp = group[timestamp_col] - group[timestamp_col].iloc[-1]
    tmp = tmp.fillna(pd.Timedelta(seconds=0))
    group["timesincecasestart"] = tmp.apply(lambda x: float(x / np.timedelta64(1, 'm'))) # m is for minutes

    group = group.sort_values(timestamp_col, ascending=True, kind='mergesort')
    group["event_nr"] = range(1, len(group) + 1)
    
    

    try:
        last_e = pd.to_datetime(
            group[
                group[activity_col].isin(
                    list(set(last_events))
                )
            ][timestamp_col].values[0]
        )
        group["time_to_event_m"] = (
            last_e - pd.to_datetime(group[timestamp_col])
        ).dt.total_seconds() / 60.0
        group["time_to_event_m"] = group["time_to_event_m"].fillna(
            pd.Timedelta(minute=0)
        )
        #     group["time_to_event_days"] = group["time_to_event_m"] / 1440
        #     group["time_to_event_wks"] = group["time_to_event_days"] / 7

        group["case_length"] = group["event_nr"].max()

        return group
    except:
        print(f"False, continue, {set(group['Case ID'])}")


def get_open_cases(date, dt_first_last_timestamps):
    return sum(
        (dt_first_last_timestamps["start_time"] <= date)
        & (dt_first_last_timestamps["end_time"] > date)
    )


In [ ]:
print("Extracting timestamp features...")
data = data.groupby(case_id_col).apply(extract_timestamp_features_2)

print("Extracting open cases...")
data = data.sort_values([timestamp_col], ascending=True, kind="mergesort").reset_index(
    drop=True
)
dt_first_last_timestamps = data.groupby(case_id_col)[timestamp_col].agg([min, max])
dt_first_last_timestamps.columns = ["start_time", "end_time"]
data["open_cases"] = data[timestamp_col].apply(
    lambda x: get_open_cases(x, dt_first_last_timestamps)
)

In [49]:
set(pdd["concept:name"])

{'Add penalty',
 'Appeal to Judge',
 'Create Fine',
 'Insert Date Appeal to Prefecture',
 'Insert Fine Notification',
 'Notify Result Appeal to Offender',
 'Payment',
 'Receive Result Appeal from Prefecture',
 'Send Appeal to Prefecture',
 'Send Fine',
 'Send for Credit Collection'}

<IPython.core.display.Javascript object>

In [40]:
case_id_col = "Case ID"
activity_col = "Activity"
resource_col = "Resource"
timestamp_col = "Complete Timestamp"
label_col = "label"
pos_label = "deviant"
neg_label = "regular"

# treatment col
treatment_col = "treatment"
pos_treatment = "treat"  # do treatment
neg_treatment = "noTreat"  # do not treat


# features for classifier
dynamic_cat_cols = ["Activity", "Resource", "lastSent", "notificationType", "dismissal"]
static_cat_cols = ["article", "vehicleClass"]
dynamic_num_cols = [
    "expense",
    "timesincelastevent",
    "timesincecasestart",
    "timesincemidnight",
    "event_nr",
    "month",
    "weekday",
    "hour",
    "open_cases",
]
static_num_cols = ["amount", "points"]


static_cols = static_cat_cols + static_num_cols + [case_id_col, label_col]
dynamic_cols = dynamic_cat_cols + dynamic_num_cols + [timestamp_col]
cat_cols = dynamic_cat_cols + static_cat_cols
freq_threshold = 10

<IPython.core.display.Javascript object>

In [64]:
datasets = ["trafficFines"]
for dataset_name in datasets:
    # dataset_manager = DatasetManager(dataset_name)
    # data = dataset_manager.read_dataset()
    dataset_name = "%s" % dataset_name
    if dataset_name == "bpic2012":
        sep = ","
    else:
        sep = ";"
    data = pd.read_csv(
        "./data/%s/%s.csv" % (dataset_name, dataset_name), sep=sep, low_memory=False
    )
    # print(set(data.Activity))

    # TO REMOVE WHITE SPACE FROM COLUMNS VALUES
    df_obj = data.select_dtypes(["object"])
    # print(data)
    data[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

<IPython.core.display.Javascript object>

In [65]:
data = data.sort_values([timestamp_col], ascending=True, kind="mergesort")
data.head()

,article,vehicleClass,amount,points,Case ID,Activity,Resource,lastSent,notificationType,dismissal,expense,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
350402,158.0,A,62.59,0.0,S38735,Create Fine,18.0,missing,missing,NIL,0.0,2000-01-01 01:00:00,60,1,5,1,0.0,0.0,1,0,deviant
352518,158.0,A,62.59,0.0,S44306,Create Fine,704.0,missing,missing,NIL,0.0,2000-01-02 01:00:00,60,1,6,1,0.0,0.0,1,1,deviant
356174,158.0,A,31.30,0.0,S49055,Create Fine,35.0,missing,missing,NIL,0.0,2000-01-02 01:00:00,60,1,6,1,0.0,0.0,1,2,regular
356180,158.0,A,31.30,0.0,S49056,Create Fine,35.0,missing,missing,NIL,0.0,2000-01-02 01:00:00,60,1,6,1,0.0,0.0,1,3,deviant
356184,158.0,A,31.30,0.0,S49057,Create Fine,35.0,missing,missing,NIL,0.0,2000-01-02 01:00:00,60,1,6,1,0.0,0.0,1,4,deviant


<IPython.core.display.Javascript object>

In [67]:
last_events = (
    data.sort_values([timestamp_col], ascending=True, kind="mergesort")
    .groupby(case_id_col)
    .last()["Activity"]
)
set(last_events)

{'Add penalty',
 'Appeal to Judge',
 'Notify Result Appeal to Offender',
 'Payment',
 'Receive Result Appeal from Prefecture',
 'Send Appeal to Prefecture'}

<IPython.core.display.Javascript object>

In [ ]:
{'Appeal to Judge',
 'Notify Result Appeal to Offender',
 'Payment',
 'Receive Result Appeal from Prefecture',
 'Send Appeal to Prefecture',
 'Send for Credit Collection'}

In [45]:
data[data["Case ID"] == "A100"]

,article,vehicleClass,amount,points,Case ID,Activity,Resource,lastSent,notificationType,dismissal,expense,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
0,157.0,A,35.0,0.0,A100,Create Fine,561.0,missing,missing,NIL,0.0,2006-08-02 01:00:00,60,8,2,1,0.0,0.0,1,11246,deviant
1,157.0,A,35.0,0.0,A100,Send Fine,561.0,missing,missing,NIL,11.0,2006-12-12 01:00:00,60,12,1,1,190080.0,190080.0,2,12380,deviant
2,157.0,A,35.0,0.0,A100,Insert Fine Notification,561.0,P,P,NIL,11.0,2007-01-15 01:00:00,60,1,0,1,48960.0,239040.0,3,12202,deviant
3,157.0,A,71.5,0.0,A100,Add penalty,561.0,P,P,NIL,11.0,2007-03-16 01:00:00,60,3,4,1,86400.0,325440.0,4,7749,deviant


<IPython.core.display.Javascript object>

In [25]:
def extract_timestamp_features_2(group):
    group[timestamp_col] = pd.to_datetime(group[timestamp_col])

    group = group.sort_values(timestamp_col, ascending=True, kind="mergesort")

#     tmp = group[timestamp_col] - group[timestamp_col].shift(-1)
#     tmp = tmp.fillna(pd.Timedelta(seconds=0))
#     group["timesincelastevent"] = tmp.apply(
#         lambda x: float(x / np.timedelta64(1, "m"))
#     )  # m is for minutes

#     tmp = group[timestamp_col] - group[timestamp_col].iloc[-1]
#     tmp = tmp.fillna(pd.Timedelta(seconds=0))
#     group["timesincecasestart"] = tmp.apply(
#         lambda x: float(x / np.timedelta64(1, "m"))
#     )  # m is for minutes

#     group = group.sort_values(timestamp_col, ascending=True, kind="mergesort")
#     group["event_nr"] = range(1, len(group) + 1)

#     group["timesincecasestart_days"] = group["timesincecasestart"] / 1440
#     group["timesincecasestart_wks"] = group["timesincecasestart_days"] / 7

#     group["timesincelastevent_days"] = group["timesincelastevent"] / 1440
#     group["timesincelastevent_wks"] = group["timesincelastevent_days"] / 7

    try:
        last_e = pd.to_datetime(
            group[
                group[activity_col].isin(
                    [
                        "Send Fine"
                    ]
                )
            ][timestamp_col].values[0]
        )
        group["time_to_event_m"] = (
            last_e - pd.to_datetime(group[timestamp_col])
        ).dt.total_seconds() / 60.0
        group["time_to_event_m"] = group["time_to_event_m"].fillna(
            pd.Timedelta(seconds=0)
        )
        #     group["time_to_event_days"] = group["time_to_event_m"] / 1440
        #     group["time_to_event_wks"] = group["time_to_event_days"] / 7

        group["case_length"] = group["event_nr"].max()

        return group
    except:
        print(f"False, continue, {set(group['Case ID'])}")


def get_open_cases(date, dt_first_last_timestamps):
    return sum(
        (dt_first_last_timestamps["start_time"] <= date)
        & (dt_first_last_timestamps["end_time"] > date)
    )


<IPython.core.display.Javascript object>

In [ ]:
print("Extracting timestamp features...")
data = data.groupby(case_id_col).apply(extract_timestamp_features_2)

print("Extracting open cases...")
data = data.sort_values([timestamp_col], ascending=True, kind="mergesort").reset_index(
    drop=True
)
dt_first_last_timestamps = data.groupby(case_id_col)[timestamp_col].agg([min, max])
dt_first_last_timestamps.columns = ["start_time", "end_time"]
data["open_cases"] = data[timestamp_col].apply(
    lambda x: get_open_cases(x, dt_first_last_timestamps)
)

In [14]:
data

,article,vehicleClass,amount,points,Case ID,Activity,Resource,lastSent,notificationType,dismissal,...,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
0,157.0,A,35.0,0.0,A100,Create Fine,561.0,missing,missing,NIL,...,2006-08-02 01:00:00,60,8,2,1,0.0,0.0,1,11246,deviant
1,157.0,A,35.0,0.0,A100,Send Fine,561.0,missing,missing,NIL,...,2006-12-12 01:00:00,60,12,1,1,190080.0,190080.0,2,12380,deviant
2,157.0,A,35.0,0.0,A100,Insert Fine Notification,561.0,P,P,NIL,...,2007-01-15 01:00:00,60,1,0,1,48960.0,239040.0,3,12202,deviant
3,157.0,A,71.5,0.0,A100,Add penalty,561.0,P,P,NIL,...,2007-03-16 01:00:00,60,3,4,1,86400.0,325440.0,4,7749,deviant
4,157.0,A,36.0,0.0,A10000,Create Fine,561.0,missing,missing,NIL,...,2007-03-09 01:00:00,60,3,4,1,0.0,0.0,1,7756,regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460551,142.0,A,262.0,0.0,V9998,Add penalty,25.0,N,P,NIL,...,2002-12-30 01:00:00,60,12,0,1,86400.0,164160.0,4,12324,deviant
460552,142.0,A,131.0,0.0,V9999,Create Fine,25.0,missing,missing,NIL,...,2002-09-07 01:00:00,60,9,5,1,0.0,0.0,1,12131,deviant
460553,142.0,A,131.0,0.0,V9999,Send Fine,25.0,missing,missing,NIL,...,2002-10-25 01:00:00,60,10,4,1,69120.0,69120.0,2,12308,deviant
460554,142.0,A,131.0,0.0,V9999,Insert Fine Notification,25.0,N,P,NIL,...,2002-11-04 01:00:00,60,11,0,1,14400.0,83520.0,3,12329,deviant


<IPython.core.display.Javascript object>

In [23]:
data[data["Case ID"] == "V9999"]

,article,vehicleClass,amount,points,Case ID,Activity,Resource,lastSent,notificationType,dismissal,expense,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
460552,142.0,A,131.0,0.0,V9999,Create Fine,25.0,missing,missing,NIL,0.00,2002-09-07 01:00:00,60,9,5,1,0.0,0.0,1,12131,deviant
460553,142.0,A,131.0,0.0,V9999,Send Fine,25.0,missing,missing,NIL,15.16,2002-10-25 01:00:00,60,10,4,1,69120.0,69120.0,2,12308,deviant
460554,142.0,A,131.0,0.0,V9999,Insert Fine Notification,25.0,N,P,NIL,15.16,2002-11-04 01:00:00,60,11,0,1,14400.0,83520.0,3,12329,deviant
460555,142.0,A,262.0,0.0,V9999,Add penalty,25.0,N,P,NIL,15.16,2003-01-03 01:00:00,60,1,4,1,86400.0,169920.0,4,12326,deviant


<IPython.core.display.Javascript object>

In [21]:
data[data["Case ID"] == "A10000"]

,article,vehicleClass,amount,points,Case ID,Activity,Resource,lastSent,notificationType,dismissal,expense,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
4,157.0,A,36.0,0.0,A10000,Create Fine,561.0,missing,missing,NIL,0.0,2007-03-09 01:00:00,60,3,4,1,0.0,0.0,1,7756,regular
5,157.0,A,36.0,0.0,A10000,Send Fine,561.0,missing,missing,NIL,13.0,2007-07-17 01:00:00,60,7,1,1,187200.0,187200.0,2,9347,regular
6,157.0,A,36.0,0.0,A10000,Insert Fine Notification,561.0,P,P,NIL,13.0,2007-08-02 01:00:00,60,8,3,1,23040.0,210240.0,3,9838,regular
7,157.0,A,74.0,0.0,A10000,Add penalty,561.0,P,P,NIL,13.0,2007-10-01 01:00:00,60,10,0,1,86400.0,296640.0,4,10875,regular
8,157.0,A,74.0,0.0,A10000,Payment,561.0,P,P,NIL,13.0,2008-09-09 01:00:00,60,9,1,1,495360.0,792000.0,5,14502,regular


<IPython.core.display.Javascript object>

In [18]:
data[data["Case ID"] == "A100"]

,article,vehicleClass,amount,points,Case ID,Activity,Resource,lastSent,notificationType,dismissal,...,Complete Timestamp,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,open_cases,label
0,157.0,A,35.0,0.0,A100,Create Fine,561.0,missing,missing,NIL,...,2006-08-02 01:00:00,60,8,2,1,0.0,0.0,1,11246,deviant
1,157.0,A,35.0,0.0,A100,Send Fine,561.0,missing,missing,NIL,...,2006-12-12 01:00:00,60,12,1,1,190080.0,190080.0,2,12380,deviant
2,157.0,A,35.0,0.0,A100,Insert Fine Notification,561.0,P,P,NIL,...,2007-01-15 01:00:00,60,1,0,1,48960.0,239040.0,3,12202,deviant
3,157.0,A,71.5,0.0,A100,Add penalty,561.0,P,P,NIL,...,2007-03-16 01:00:00,60,3,4,1,86400.0,325440.0,4,7749,deviant


<IPython.core.display.Javascript object>

In [17]:
set(data["Activity"])

{'Add penalty',
 'Appeal to Judge',
 'Create Fine',
 'Insert Date Appeal to Prefecture',
 'Insert Fine Notification',
 'Notify Result Appeal to Offender',
 'Payment',
 'Receive Result Appeal from Prefecture',
 'Send Appeal to Prefecture',
 'Send Fine'}

<IPython.core.display.Javascript object>

In [ ]:
# min cols
case_id_col = "Case ID"
activity_col = "Activity"
resource_col = "org:resource"
timestamp_col = "time:timestamp"
# label/outcome col
label_col = "label"
neg_label = "regular"  # negative outcome that we don't need to predict
pos_label = "deviant"  # positive outcome that will be predicted
# treatment col
treatment_col = "treatment"
pos_treatment = "treat"  # do treatment
neg_treatment = "noTreat"  # do not treat

# features for classifier
dynamic_cat_cols = [
    "Activity",
    "org:resource",
    "Action",
    "EventOrigin",
    "lifecycle:transition",
    "Accepted",
    "Selected",
]
static_cat_cols = [
    "ApplicationType",
    "LoanGoal",
]  # static attributes, no need for predicting in suffix predictions
dynamic_num_cols = [
    "FirstWithdrawalAmount",
    "MonthlyCost",
    "NumberOfTerms",
    "OfferedAmount",
    "CreditScore",
    "timesincelastevent",
    "timesincecasestart",
    "timesincemidnight",
    "event_nr",
    "month",
    "weekday",
    "hour",
    "open_cases",
]
static_num_cols = [
    "NumberOfOffers",
    "RequestedAmount",
]  # static attributes, no need for predicting in suffix predictions


static_cols = static_cat_cols + static_num_cols + [case_id_col, label_col]
dynamic_cols = dynamic_cat_cols + dynamic_num_cols + [timestamp_col]
cat_cols = dynamic_cat_cols + static_cat_cols
freq_threshold = 10